In [37]:
import spotipy
import spotipy.oauth2
import spotipy.util as util
import pandas as pd
import numpy as np

import collections

In [27]:
dat = pd.read_csv('combined_all_countries_1.csv', encoding='latin-1')

In [28]:
dat.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace = True)

In [30]:
dat.head(3)

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,...,no,hu,pt,is,bo,au,hk,tw,my,ee
0,0.58100,https://api.spotify.com/v1/audio-analysis/7qiZ...,0.825,233713,0.652,True,0.00000,1,0.0931,-3.183,...,True,True,True,True,True,True,True,True,True,True
1,0.00902,https://api.spotify.com/v1/audio-analysis/12VW...,0.785,173987,0.617,True,0.00246,1,0.3510,-5.871,...,True,True,True,True,True,True,True,True,True,True
2,0.41500,https://api.spotify.com/v1/audio-analysis/7crM...,0.736,245507,0.541,True,0.00000,8,0.1100,-5.597,...,True,True,True,True,True,True,True,True,True,True


In [45]:
dat.columns

Index(['acousticness', 'analysis_url', 'danceability', 'duration_ms', 'energy',
       'id', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'track_href', 'type', 'uri',
       'valence', 'popularity', 'song', 'artist', 'genre', 'sv', 'cr', 'de',
       'co', 'dk', 'at', 'pa', 'es', 'gb', 'sk', 'global', 'nl', 'br', 'cl',
       'ec', 'lt', 'us', 'it', 'be', 'lu', 'gr', 'mx', 'uy', 'ca', 'jp', 'sg',
       'tr', 'cz', 'lv', 'pe', 'ch', 'se', 'ar', 'gt', 'py', 'pl', 'ph', 'nz',
       'fr', 'fi', 'hn', 'ie', 'do', 'no', 'hu', 'pt', 'is', 'bo', 'au', 'hk',
       'tw', 'my', 'ee'],
      dtype='object')

In [43]:
duplicates = [item for item, count in collections.Counter(dat.song).items() if count > 1]
print(len(duplicates))
duplicates

19


['Perfect',
 "Don't",
 'Wrecking Ball',
 'Love Myself',
 'Reality - Radio Edit',
 'Skinny Love',
 'Ghost',
 'Downtown (feat. Melle Mel, Grandmaster Caz, Kool Moe Dee & Eric Nally)',
 'Sweater Weather',
 'Wild Ones (feat. Sia)',
 'Seven Nation Army',
 'Bad Romance',
 'Shed a Light',
 'This Town',
 'Bad Things (with Camila Cabello)',
 'Now Or Never',
 'Malibu',
 'Too Good At Goodbyes',
 'Cold (feat. Future)']

The following songs are 'technically duplicates' but have very slight differences in their features, as in they are the same songs by the same artist but have minor changes in energy/acousticness etc... (which is why the uri for each song is unique)

Remove or keep?

Wrecking Ball

Love Myself

Reality - Radio Edit

Ghost

Downtown (feat. Melle Mel, Grandmaster Caz, Kool Moe Dee & Eric Nally) 

Sweater Weather - the differences are kinda non-trivial in this one.

Wild Ones (feat. Sia)

Seven Nation Army

Bad Romance

Shed a Light

This Town

Bad Things (with Camila Cabello)

Now Or Never

Malibu

Too Good At Goodbyes

Cold (feat. Future)

In [92]:
dat[dat.song == 'Downtown (feat. Melle Mel, Grandmaster Caz, Kool Moe Dee & Eric Nally)']
dat.drop(dat.index[])

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,...,no,hu,pt,is,bo,au,hk,tw,my,ee
305,0.0916,https://api.spotify.com/v1/audio-analysis/0lLD...,0.828,292593,0.609,False,0.000001,1,0.291,-4.402,...,False,False,False,False,False,True,False,False,False,False
419,0.0916,https://api.spotify.com/v1/audio-analysis/5pfJ...,0.828,292593,0.609,False,0.000001,1,0.291,-4.402,...,False,False,False,False,False,True,False,False,False,False


In [187]:
copy = dat.copy()

In [97]:
world = pd.read_csv('data.csv')

In [98]:
world.head(2)

,Position,Track Name,Artist,Streams,URL,Date,Region
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,2017-01-01,ec
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,2017-01-01,ec


In [105]:
pos_1_ever = set(world[world.Position == 1]['Track Name'])
print(len(pos_1_ever)) #257 songs that have ever been top of the list

257


In [107]:
region_names = list(set(world.Region))

In [108]:
region_names[0] + '_top'

'py_top'

In [149]:
set(world[(world['Position'] == 1) & (world['Region'] == 'no')]['Track Name'])

{'All Falls Down',
 'All I Want for Christmas Is You',
 'Alone',
 'Despacito (Featuring Daddy Yankee)',
 'Despacito - Remix',
 'Friends (with BloodPop®)',
 'Fy Faen',
 'Home for Christmas',
 "It Ain't Me (with Selena Gomez)",
 'Rett i foret',
 'River (feat. Ed Sheeran)',
 'Shape of You',
 'Such A Boy',
 'Symphony (feat. Zara Larsson)',
 'Think Before I Talk',
 'Without You (feat. Sandro Cavazza)',
 'rockstar'}

In [150]:
copy.song[0]

'Shape of You'

In [188]:
for i in region_names:
    new_col = i + '_top' #each country now has column to state if song was ever top song in that country
    col_values = []
    pos_1_ever = set(world[(world['Position'] == 1) & (world['Region'] == i)]['Track Name']) # 'i' is region 
    for j in range(0, len(copy)):
        if copy.song[j] in pos_1_ever:
            col_values.append(True)
        else:
            col_values.append(False)
    copy[new_col] = col_values

In [190]:
copy[18:22] #successfully added columns

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,...,pt_top,ca_top,co_top,nz_top,tr_top,cz_top,hn_top,be_top,id_top,bo_top
18,0.00801,https://api.spotify.com/v1/audio-analysis/32Ol...,0.856,269667,0.609,False,0.000081,0,0.0344,-7.223,...,False,False,False,False,False,False,False,False,False,False
19,0.14100,https://api.spotify.com/v1/audio-analysis/023O...,0.780,180560,0.680,False,0.000012,4,0.1380,-6.081,...,False,False,False,False,False,False,False,False,False,False
20,0.28700,https://api.spotify.com/v1/audio-analysis/5kqI...,0.765,237300,0.473,True,0.000000,10,0.3910,-5.829,...,False,False,False,False,False,False,False,False,False,False
21,0.20000,https://api.spotify.com/v1/audio-analysis/7CUY...,0.618,228200,0.793,True,0.000000,2,0.1060,-4.777,...,True,True,True,True,True,True,True,True,True,True


In [229]:
copy2 = copy.copy() #fresh copy

In [227]:
a = world[world['Region'] == 'ee']['Track Name'].value_counts()
set(a[a > 100].index)

{'All Time Low',
 'Attention',
 'Believer',
 'Call On Me - Ryan Riback Extended Remix',
 'Castle on the Hill',
 'Congratulations',
 'Despacito (Featuring Daddy Yankee)',
 'Despacito - Remix',
 'Galway Girl',
 'HUMBLE.',
 'I Don’t Wanna Live Forever (Fifty Shades Darker) - From "Fifty Shades Darker (Original Motion Picture Soundtrack)"',
 "I'm the One",
 'Issues',
 "It Ain't Me (with Selena Gomez)",
 'Mama',
 'Mask Off',
 'More Than You Know',
 'No Promises (feat. Demi Lovato)',
 'Now Or Never',
 'Paris',
 'Pretty Girl - Cheat Codes X CADE Remix',
 'Rockabye (feat. Sean Paul & Anne-Marie)',
 'Scared to Be Lonely',
 'Shape of You',
 'Slide',
 'Slow Hands',
 'Solo Dance',
 'Something Just Like This',
 'Stay (with Alessia Cara)',
 'Swalla (feat. Nicki Minaj & Ty Dolla $ign)',
 'Symphony (feat. Zara Larsson)',
 "That's What I Like",
 "There's Nothing Holdin' Me Back",
 'Thunder',
 'Unforgettable',
 'Weak',
 'XO TOUR Llif3'}

In [233]:
for i in region_names:
    new_col = i + '_100' #each country now has column to state if song was 100 days popular in that country
    col_values = []
    ever_show = set(world[world['Region'] == i]['Track Name'])
    song_frequencies = world[world['Region'] == i]['Track Name'].value_counts() #get frequencies of each song in this region
    days_100 = set(song_frequencies[song_frequencies > 100].index) # 'i' is region, get all songs >= 100 frequencies 
    for j in range(0, len(copy)):
        if copy2.song[j] not in ever_show:
            col_values.append(False) #it looks redundant now, was using for testing
        elif copy2.song[j] in days_100:
            col_values.append(True)
        else:
            col_values.append(False)
    copy2[new_col] = col_values

In [234]:
copy2.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,...,pt_100,ca_100,co_100,nz_100,tr_100,cz_100,hn_100,be_100,id_100,bo_100
0,0.58100,https://api.spotify.com/v1/audio-analysis/7qiZ...,0.825,233713,0.652,True,0.00000,1,0.0931,-3.183,...,True,True,True,True,True,True,True,True,True,True
1,0.00902,https://api.spotify.com/v1/audio-analysis/12VW...,0.785,173987,0.617,True,0.00246,1,0.3510,-5.871,...,True,True,True,True,True,True,True,True,False,True
2,0.41500,https://api.spotify.com/v1/audio-analysis/7crM...,0.736,245507,0.541,True,0.00000,8,0.1100,-5.597,...,True,True,True,True,True,True,True,True,True,True
3,0.00346,https://api.spotify.com/v1/audio-analysis/4vS8...,0.723,176561,0.809,True,0.00123,7,0.5650,-3.081,...,True,False,True,True,True,True,False,True,False,False
4,0.47400,https://api.spotify.com/v1/audio-analysis/34gC...,0.781,281560,0.445,True,0.00000,2,0.1840,-6.061,...,True,True,True,True,False,True,True,True,True,True


In [ ]:
copy2.to_csv('combined_all_countries_2.csv') #all columns 